모듈 다운로드..

In [ ]:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
# %pip install transformers datasets accelerate

In [ ]:
# %pip install pandas scikit-learn tqdm

In [ ]:
# %pip install ipywidgets

In [ ]:
# %pip install "huggingface_hub[hf_xet]"

In [ ]:
import torch

print(f"PyTorch 버전: {torch.__version__}")
print(f"GPU 사용 가능 여부: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")
    device = torch.device("cuda")
else:
    print("GPU를 찾을 수 없어 CPU를 사용합니다.")
    device = torch.device("cpu")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAME = "snunlp/KR-FinBert-SC"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 모델 로드 및 GPU 할당
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3)
model.to(device)  # 이 코드가 모델을 GPU로 보냅니다.

# 모델 추가학습 코드

FMKorea

In [ ]:
import json
import os
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments
)

# 1. 설정 및 장치 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"현재 사용 중인 장치: {device}")

# 2. 리스트 (생략 없이 그대로 사용)
# 업데이트된 Fear 리스트
fear_words = [
    '공포', '하락', '인버스', '패닉', '셀링', '손절', '풀숏', '숏', '불안', '위험', '회피', 
    '리스크', '침체', '변동성', '마이너스', '물타기', '패닉셀', '매도', '존버', '익절', 
    '조정', '도망', '포기', '이탈', '환장', '한강', '자살', '수온', '음전', '설거지', 
    '개미털이', '공매도', '떡락', '탈주', '금리인상', '추락', '피바다', '반등없음', '쫄림', 
    '절망', '멘붕', '현금확보', '지옥', '외인매도', '기관매도', '양아치', '사기', '물림', 
    '손실확정', '악재', '청산', '잡주', '빠지', '망하', '좆', '새끼', '죽', '반대', 
    '힘들', '조심', '급락', '나쁘', '당하', '병신', '버블', '탈출', '끝물', '구조대', 
    '물리', '관세', '밀리', '내리', '떨어지', '음봉', '하향', '부진', '부담', '우려', 
    '실망', '비싸', '관망', '박스권', '고평가', '거품',
    '팔', '지랄', '좃', '던지', '버스', '뒤지', '처박', '실패', '시발', '무섭', '공포장'
]
greed_words = [
    '탐욕', '레버리지', '풀롱', '롱', '수출호재', '호재', '기대', '떡상', '드가자', '가즈아', 
    '매수', '신고가', '고점', '몰빵', '상승', '폭등', '불장', '랠리', 'FOMO', '포모', 
    '효자', '불타기', '과열', '쭉쭉', '급등주', '급등', '추매', '최고', '대장', '사랑', 
    '감사', '갓전자', '갓하이닉스', '갓현대', '갓차', '개미승리', '상방확정', '호재반영', 
    '롱진입', '외인매수', '기관매수', '마진확대', '우상향', '폭주기관차', '상승장', '행복', 
    '안착', '성공', '부럽', '커피값', '저녁값', '소고기', '광기', '회복', '영끌', '빚투', 
    '축하', '와우', '대박', '추격매수', '전고점', '매집중', '돌파', '수익', '반등', '양봉', 
    '실현', '베팅', '홀딩', '기대감', '차익', '부자', '강하', '수혜', '강세', '성장', 
    '사이클', '수급',
    '오르', '이익', '기회', '모으', '수주', '상방', '가치', '수익중'
]

# 3. 데이터 로드 및 라벨링 함수
def prepare_dataset(jsonl_path):
    if not os.path.exists(jsonl_path):
        raise FileNotFoundError(f"파일을 찾을 수 없습니다: {jsonl_path}")
    
    data = []
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line in f:
            post = json.loads(line)
            texts = [post.get('title', ''), post.get('content', '')]
            for c in post.get('comments', []) or []:
                if isinstance(c, dict): texts.append(c.get('comment', ''))
            
            for text in texts:
                text = str(text).strip()
                if len(text) < 5: continue
                f_score = sum(1 for w in fear_words if w in text)
                g_score = sum(1 for w in greed_words if w in text)
                label = 0 if f_score > g_score else (2 if g_score > f_score else 1)
                data.append({'text': text, 'label': label})
    return pd.DataFrame(data).drop_duplicates()

# 4. 데이터셋 클래스 정의
class StockDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

# 5. 실행 로직 (이 순서가 중요합니다)
print("데이터셋 준비 중...")
df = prepare_dataset('../data/fmkorea_hynix_hot_posts.jsonl')
train_df, val_df = train_test_split(df, test_size=0.1, stratify=df['label'], random_state=42)

MODEL_NAME = "snunlp/KR-FinBert-SC"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print("토크나이징 중...")
train_encodings = tokenizer(train_df['text'].tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_df['text'].tolist(), truncation=True, padding=True, max_length=128)

# 변수 생성 확인
train_dataset = StockDataset(train_encodings, train_df['label'].tolist())
val_dataset = StockDataset(val_encodings, val_df['label'].tolist())

# 6. 모델 학습
print("모델 로드 중...")
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3).to(device)

training_args = TrainingArguments(
    output_dir='./stock_model_checkpoints',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    fp16=True if torch.cuda.is_available() else False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset, # 이제 메모리에 확실히 존재함
    eval_dataset=val_dataset,
)

print("학습 시작...")
trainer.train()

# 모델 저장
model.save_pretrained("./finetuned_stock_bert")
tokenizer.save_pretrained("./finetuned_stock_bert")
print("모델 저장 완료: ./finetuned_stock_bert")

현재 사용 중인 장치: cuda
데이터셋 준비 중...
토크나이징 중...
모델 로드 중...
학습 시작...


Epoch,Training Loss,Validation Loss
1,0.273100,0.184250
2,0.138800,0.149888
3,0.084900,0.149835


모델 저장 완료: ./finetuned_stock_bert


## 추가학습된 모델로 데이터 공포/탐욕 지수 생성

In [31]:
import json
import os
from collections import defaultdict
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

# =========================
# 1. 설정 및 경로
# =========================
JSONL_PATH = r"..\data\fmkorea_hynix_hot_posts.jsonl"
OUT_DIR = r"..\output"
os.makedirs(OUT_DIR, exist_ok=True)
OUT_DAILY_CSV = os.path.join(OUT_DIR, "daily_fng_balanced.csv")
OUT_WEEKLY_CSV = os.path.join(OUT_DIR, "weekly_fng_balanced.csv")

START_DATE = "2025-01-14"
END_DATE   = "2026-01-14"

MODEL_NAME = "./finetuned_stock_bert" 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# =========================
# 2. 모델 로드 및 추론 로직 (중립 문턱 최적화)
# =========================
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to(DEVICE)
model.eval()

def probs_to_fng(logits):
    probs = torch.softmax(logits, dim=-1).squeeze(0).cpu().numpy()
    f_p, n_p, g_p = probs[0], probs[1], probs[2]
    
    # [조정] 중립 임계값을 0.60으로 살짝 상향 (0.5는 너무 예민하여 노이즈가 심함)
    neutral_threshold = 0.60 
    
    if n_p > neutral_threshold:
        label, score = "neutral", 0.0
    elif f_p > g_p:
        label, score = "fear", -1.0
    else:
        label, score = "greed", 1.0
        
    return label, score

def infer_one(text: str):
    if not text or not str(text).strip():
        return None
    inputs = tokenizer(str(text), return_tensors="pt", truncation=True, max_length=128).to(DEVICE)
    with torch.no_grad():
        logits = model(**inputs).logits
    return probs_to_fng(logits)

# =========================
# 3. 데이터 집계
# =========================
start_d = pd.to_datetime(START_DATE).date()
end_d   = pd.to_datetime(END_DATE).date()

daily_stats = defaultdict(lambda: {"fear": 0, "neutral": 0, "greed": 0, "total_w": 0.0})

print(f"분석 시작 (장치: {DEVICE})...")

with open(JSONL_PATH, "r", encoding="utf-8") as f:
    lines = f.readlines()
    for line in tqdm(lines):
        post = json.loads(line)
        d = pd.to_datetime(post.get("date"), errors="coerce")
        if pd.isna(d): continue
        d_date = d.date()
        if d_date < start_d or d_date > end_d: continue

        # 제목 가중치를 1.5배로 약간 하향 (2.0은 변동성을 너무 키움)
        texts = [('제목', post.get("title", "")), ('본문', post.get("content", ""))]
        for c in post.get("comments", []) or []:
            if isinstance(c, dict):
                texts.append(('댓글', c.get("comment", "")))

        for t_type, t in texts:
            out = infer_one(t)
            if out is None: continue
            label, s = out
            
            weight = 1.5 if t_type == '제목' else 1.0
            daily_stats[d_date][label] += weight
            daily_stats[d_date]["total_w"] += weight

# =========================
# 4. 결과 산출 및 평활화 (Smoothing)
# =========================
def calculate_raw_index(stats):
    f, n, g = stats["fear"], stats["neutral"], stats["greed"]
    tw = stats["total_w"]
    if tw == 0: return 50.0, 0.0
    
    active_sum = f + g
    if active_sum == 0: return 50.0, 0.0
    
    sentiment_direction = (g - f) / active_sum
    emotion_density = np.sqrt(active_sum / tw) 
    
    # [조정] 증폭 계수를 2.5로 하향 (3.5는 너무 들쭉날쭉함)
    scaled_score = np.tanh(sentiment_direction * emotion_density * 2.5)
    fng_index = (scaled_score + 1.0) * 50.0
    
    return fng_index, emotion_density

# 일일 원본 데이터 생성
daily_rows = []
for d in pd.date_range(start_d, end_d, freq="D").date:
    if d not in daily_stats: continue
    idx, dens = calculate_raw_index(daily_stats[d])
    daily_rows.append({
        "date": d,
        "fng_raw": idx,
        "emotion_density": dens
    })

df_daily = pd.DataFrame(daily_rows)

# [핵심] 3일 이동 평균 적용: 일일 노이즈를 제거하여 흐름을 부드럽게 만듦
df_daily['fng_index'] = df_daily['fng_raw'].rolling(window=3, min_periods=1, center=True).mean().round(2)

# 저장
df_daily[['date', 'fng_index', 'emotion_density']].to_csv(OUT_DAILY_CSV, index=False, encoding="utf-8-sig")

# --- 주간 데이터 처리 ---
df_daily['date'] = pd.to_datetime(df_daily['date'])
df_weekly = df_daily.resample('W-MON', on='date').mean().reset_index()
df_weekly['fng_index'] = df_weekly['fng_index'].round(2)
df_weekly.to_csv(OUT_WEEKLY_CSV, index=False, encoding="utf-8-sig")

print(f"\n[완료] 지수 산출 결과가 저장되었습니다.")

분석 시작 (장치: cuda)...


100%|██████████| 1100/1100 [01:31<00:00, 11.97it/s]


[완료] 지수 산출 결과가 저장되었습니다.


# 모델 추가학습 및 디시인사이드 최종 지수 산출 

In [41]:
import os
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments
)

# 1. 설정 및 장치 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"현재 사용 중인 장치: {device}")

# 2. 감성 단어 리스트 (최신 업데이트 버전)
fear_words = [
    '공포', '하락', '인버스', '패닉', '셀링', '손절', '풀숏', '숏', '불안', '위험', '회피', 
    '리스크', '침체', '변동성', '마이너스', '물타기', '패닉셀', '매도', '존버', '익절', 
    '조정', '도망', '포기', '이탈', '환장', '한강', '자살', '수온', '음전', '설거지', 
    '개미털이', '공매도', '떡락', '탈주', '금리인상', '추락', '피바다', '반등없음', '쫄림', 
    '절망', '멘붕', '현금확보', '지옥', '외인매도', '기관매도', '양아치', '사기', '물림', 
    '손실확정', '악재', '청산', '잡주', '빠지', '망하', '좆', '새끼', '죽', '반대', 
    '힘들', '조심', '급락', '나쁘', '당하', '병신', '버블', '탈출', '끝물', '구조대', 
    '물리', '관세', '밀리', '내리', '떨어지', '음봉', '하향', '부진', '부담', '우려', 
    '실망', '비싸', '관망', '박스권', '고평가', '거품', '팔', '지랄', '좃', '던지', '버스', 
    '뒤지', '처박', '실패', '시발', '무섭', '공포장',
    '폭락', '박살', '꼬라지', '무너짐', '손실중'
]
greed_words = [
    '탐욕', '레버리지', '풀롱', '롱', '수출호재', '호재', '기대', '떡상', '드가자', '가즈아', 
    '매수', '신고가', '고점', '몰빵', '상승', '폭등', '불장', '랠리', 'FOMO', '포모', 
    '효자', '불타기', '과열', '쭉쭉', '급등주', '급등', '추매', '최고', '대장', '사랑', 
    '감사', '갓전자', '갓하이닉스', '갓현대', '갓차', '개미승리', '상방확정', '호재반영', 
    '롱진입', '외인매수', '기관매수', '마진확대', '우상향', '폭주기관차', '상승장', '행복', 
    '안착', '성공', '부럽', '커피값', '저녁값', '소고기', '광기', '회복', '영끌', '빚투', 
    '축하', '와우', '대박', '추격매수', '전고점', '매집중', '돌파', '수익', '반등', '양봉', 
    '실현', '베팅', '홀딩', '기대감', '차익', '부자', '강하', '수혜', '강세', '성장', 
    '사이클', '수급', '오르', '이익', '기회', '모으', '수주', '상방', '가치', '수익중',
    '갓하이닉스', '불기둥', '추가상승'
]

# 3. 데이터 로드 및 라벨링 함수 (CSV 버전)
def prepare_dataset_from_csv(csv_path):
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"파일을 찾을 수 없습니다: {csv_path}")
    
    # CSV 읽기 (인코딩 주의)
    df_raw = pd.read_csv(csv_path, encoding='utf-8-sig')
    
    data = []
    print("CSV 데이터 라벨링 중...")
    for _, row in df_raw.iterrows():
        # 제목과 본문 추출 및 결합
        title = str(row.get('title', '')).strip()
        content = str(row.get('content', '')).strip()
        
        texts = [title, content]
        
        for text in texts:
            if len(text) < 5: continue # 너무 짧은 텍스트 제외
            
            f_score = sum(1 for w in fear_words if w in text)
            g_score = sum(1 for w in greed_words if w in text)
            
            # 라벨링: 0(공포), 1(중립), 2(탐욕)
            label = 0 if f_score > g_score else (2 if g_score > f_score else 1)
            data.append({'text': text, 'label': label})
            
    return pd.DataFrame(data).drop_duplicates()

# 4. 데이터셋 클래스 정의
class StockDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

# 5. 실행 로직
INPUT_CSV = "../data/posts_hynix_with_comment.csv" # 입력 파일 경로

print("데이터셋 준비 중...")
df = prepare_dataset_from_csv(INPUT_CSV)
train_df, val_df = train_test_split(df, test_size=0.1, stratify=df['label'], random_state=42)

MODEL_NAME = "snunlp/KR-FinBert-SC"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

print("토크나이징 중...")
train_encodings = tokenizer(train_df['text'].tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_df['text'].tolist(), truncation=True, padding=True, max_length=128)

train_dataset = StockDataset(train_encodings, train_df['label'].tolist())
val_dataset = StockDataset(val_encodings, val_df['label'].tolist())

# 6. 모델 학습 설정
print("모델 로드 중...")
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3).to(device)

training_args = TrainingArguments(
    output_dir='./stock_model_checkpoints',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    fp16=True if torch.cuda.is_available() else False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

print("학습 시작...")
trainer.train()

# 모델 저장
save_path = "./finetuned_stock_bert"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)
print(f"모델 저장 완료: {save_path}")

현재 사용 중인 장치: cuda
데이터셋 준비 중...
CSV 데이터 라벨링 중...
토크나이징 중...
모델 로드 중...
학습 시작...


Epoch,Training Loss,Validation Loss
1,No log,0.458737
2,0.603800,0.347841
3,0.244700,0.408167


모델 저장 완료: ./finetuned_stock_bert


In [42]:
import os
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm
from collections import defaultdict
from datetime import datetime
import re

# =========================
# 1. 설정 및 경로
# =========================
# 분석할 파일 경로 (삼성전자의 경우 posts_samsung_with_comment.csv 등으로 변경 가능)
CSV_PATH = r"..\data\posts_hynix_with_comment.csv" 
OUT_DIR = r"..\output"
os.makedirs(OUT_DIR, exist_ok=True)

# 저장될 파일명 (종목에 따라 이름을 바꿔서 저장하세요)
OUT_DAILY_CSV = os.path.join(OUT_DIR, "hynix_fng_balanced.csv")

# 학습시킨 모델 경로
MODEL_NAME = "./finetuned_stock_bert" 
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# =========================
# 2. 날짜 통일 함수 (모든 형식 대응)
# =========================
def parse_ultimate_date(date_str):
    date_str = str(date_str).strip()
    
    # 1. 시간 형식 (18:31 등)은 분석 제외
    if re.match(r'^\d{1,2}:\d{2}$', date_str):
        return None

    # 2. '작년' 키워드 처리
    if '작년' in date_str:
        return datetime(2025, 12, 31).date()

    # 3. YY.MM.DD 형식 (예: 25.12.31) 처리
    long_date_match = re.search(r'(\d{2,4})\.(\d{1,2})\.(\d{1,2})', date_str)
    if long_date_match:
        year = int(long_date_match.group(1))
        month = int(long_date_match.group(2))
        day = int(long_date_match.group(3))
        
        if year < 100: year += 2000 # 25 -> 2025 변환
        try:
            return datetime(year, month, day).date()
        except ValueError:
            return None

    # 4. MM.DD 형식 (예: 01.12) 처리 -> 2026년으로 고정
    short_date_match = re.search(r'^(\d{1,2})\.(\d{1,2})$', date_str)
    if short_date_match:
        month = int(short_date_match.group(1))
        day = int(short_date_match.group(2))
        try:
            return datetime(2026, month, day).date()
        except ValueError:
            return None
    
    # 5. 기타 표준 날짜 형식 시도
    try:
        dt = pd.to_datetime(date_str, errors='coerce')
        if pd.notna(dt):
            return dt.date()
    except:
        pass
        
    return None

# =========================
# 3. 모델 로드 및 추론 로직
# =========================
print(f"모델 로딩 중... (장치: {DEVICE})")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to(DEVICE)
model.eval()

def infer_one(text: str):
    if not text or len(str(text)) < 2: return None
    
    inputs = tokenizer(str(text), return_tensors="pt", truncation=True, max_length=128).to(DEVICE)
    with torch.no_grad():
        logits = model(**inputs).logits
    
    probs = torch.softmax(logits, dim=-1).squeeze(0).cpu().numpy()
    
    # 레이블 정의: 0(Fear), 1(Neutral), 2(Greed)
    # 중립(Neutral) 확률이 60%를 넘으면 중립으로 처리하여 노이즈 제거
    if probs[1] > 0.60: 
        return "neutral"
    return "fear" if probs[0] > probs[2] else "greed"

# =========================
# 4. 데이터 로드 및 일별 집계
# =========================
if not os.path.exists(CSV_PATH):
    print(f"파일을 찾을 수 없습니다: {CSV_PATH}")
else:
    df_raw = pd.read_csv(CSV_PATH, encoding='utf-8-sig')
    daily_stats = defaultdict(lambda: {"fear": 0, "neutral": 0, "greed": 0, "total_w": 0.0})

    print("데이터 분석 및 지수 산출 시작...")
    for _, row in tqdm(df_raw.iterrows(), total=len(df_raw)):
        # 날짜 정제
        d_date = parse_ultimate_date(row.get('date', ''))
        if d_date is None: continue # 시간 형식 등은 건너뜀
        
        # 제목과 본문 텍스트 추출
        texts = [('제목', row.get("title", "")), ('본문', row.get("content", ""))]
        
        for t_type, t in texts:
            label = infer_one(t)
            if label:
                # 제목은 주가 영향력이 크므로 1.5배 가중치 부여
                weight = 1.5 if t_type == '제목' else 1.0
                daily_stats[d_date][label] += weight
                daily_stats[d_date]["total_w"] += weight

    # =========================
    # 5. 결과 계산 및 저장
    # =========================
    daily_rows = []
    # 날짜 순으로 정렬하여 계산
    for d in sorted(daily_stats.keys()):
        stats = daily_stats[d]
        f, g, n, tw = stats["fear"], stats["greed"], stats["neutral"], stats["total_w"]
        
        # 감정 데이터가 없는 날은 중립(50점) 처리
        if tw == 0 or (f + g) == 0:
            idx, dens = 50.0, 0.0
        else:
            # 심리 방향성 (-1 ~ 1)
            sentiment_direction = (g - f) / (f + g)
            # 감정 밀도: 전체 글 중 감정 섞인 글의 비중
            emotion_density = np.sqrt((f + g) / tw)
            
            # 최종 지수 (0 ~ 100)
            scaled_score = np.tanh(sentiment_direction * emotion_density * 2.5)
            idx = (scaled_score + 1.0) * 50.0
            dens = emotion_density

        daily_rows.append({
            "date": d,
            "fng_raw": round(idx, 2),
            "emotion_density": round(dens, 4)
        })

    if daily_rows:
        df_daily = pd.DataFrame(daily_rows)
        
        # [추가] 3일 이동 평균(Rolling)을 적용하여 차트를 부드럽게 만듦
        df_daily['fng_index'] = df_daily['fng_raw'].rolling(window=3, min_periods=1, center=True).mean().round(2)

        # 필요한 컬럼만 저장
        df_daily[['date', 'fng_index', 'emotion_density']].to_csv(OUT_DAILY_CSV, index=False, encoding="utf-8-sig")

        print(f"\n[완료] 일별 지수 생성 성공!")
        print(f"- 저장 경로: {OUT_DAILY_CSV}")
        print(f"- 총 분석 일수: {len(df_daily)}일")
    else:
        print("\n[알림] 조건에 맞는 데이터가 없어 결과가 저장되지 않았습니다.")

모델 로딩 중... (장치: cuda)
데이터 분석 및 지수 산출 시작...


100%|██████████| 3367/3367 [00:27<00:00, 120.62it/s]


[완료] 일별 지수 생성 성공!
- 저장 경로: ..\output\hynix_fng_balanced.csv
- 총 분석 일수: 320일


# FMkorea 제목, 본문, 댓글 리스트 토큰화

In [27]:
import json
import os
import pandas as pd
from kiwipiepy import Kiwi
from tqdm import tqdm
from collections import Counter

# 1. Kiwi 초기화
kiwi = Kiwi(num_workers=0)

# 2. 추출할 의미 있는 품사 정의 (실질 형태소)
# NNG(일반 명사), NNP(고유 명사), VV(동사), VA(형용사), XR(어근), SL(외국어)
MEANINGFUL_TAGS = {'NNG', 'NNP', 'VV', 'VA', 'XR', 'SL'}

def tokenize_meaningful_only(input_path, output_csv, vocab_csv):
    all_processed_data = []
    word_counter = Counter() # 단어 중복 빈도 체크용
    
    print("파일 로드 중...")
    with open(input_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    print("의미 단어 추출 중 (Kiwi)...")
    for line in tqdm(lines):
        try:
            post = json.loads(line)
            texts = [('제목', post.get('title', '')), ('본문', post.get('content', ''))]
            for c in post.get('comments', []) or []:
                if isinstance(c, dict): texts.append(('댓글', c.get('comment', '')))

            for content_type, text in texts:
                if not text or len(str(text).strip()) < 2: continue
                
                # Kiwi 토큰화
                tokens = kiwi.tokenize(text)
                
                # 의미 있는 품사만 필터링하여 단어 리스트 생성
                meaningful_words = [t.form for t in tokens if t.tag in MEANINGFUL_TAGS]
                
                # 전체 단어 빈도수 업데이트 (얼마나 겹치는지 확인용)
                word_counter.update(meaningful_words)
                
                # 결과 정리
                meaningful_str = " ".join(meaningful_words)
                all_processed_data.append({
                    "date": post.get("date", ""),
                    "type": content_type,
                    "original_text": text.replace("\n", " "),
                    "meaningful_words": meaningful_str
                })
        except Exception:
            continue

    # 3-1. 추출 데이터 저장 (원문 + 의미 단어)
    df = pd.DataFrame(all_processed_data)
    df.to_csv(output_csv, index=False, encoding="utf-8-sig")
    
    # 3-2. 단어 빈도 데이터 저장 (얼마나 겹치는지 보여주는 리스트)
    vocab_df = pd.DataFrame(word_counter.most_common(), columns=['word', 'count'])
    vocab_df.to_csv(vocab_csv, index=False, encoding="utf-8-sig")
    
    print(f"\n[완료]")
    print(f"- 추출 결과 저장됨: {output_csv}")
    print(f"- 단어 빈도(중복) 리스트 저장됨: {vocab_csv}")
    print(f"- 고유 의미 단어 수: {len(vocab_df)}개")

# 실행
JSONL_PATH = "../data/fmkorea_hynix_hot_posts.jsonl"
RESULT_CSV = "fmkorea_meaningful_results.csv"
VOCAB_CSV = "fmkorea_word_counts.csv"

tokenize_meaningful_only(JSONL_PATH, RESULT_CSV, VOCAB_CSV)

C:\Users\Jeon\AppData\Local\Temp\ipykernel_26068\3261695241.py:9: DeprecationWarning: behavior of `num_workers=0` is changed since v0.21.0. If you want to keep the previous behavior, please set `num_workers=-1`.
  kiwi = Kiwi(num_workers=0)


파일 로드 중...
의미 단어 추출 중 (Kiwi)...


100%|██████████| 1100/1100 [00:12<00:00, 89.89it/s]



[완료]
- 추출 결과 저장됨: fmkorea_meaningful_results.csv
- 단어 빈도(중복) 리스트 저장됨: fmkorea_word_counts.csv
- 고유 의미 단어 수: 15440개


# 디시인사이드 토큰화

In [40]:
import os
import pandas as pd
from kiwipiepy import Kiwi
from tqdm import tqdm
from collections import Counter

# 1. Kiwi 초기화
kiwi = Kiwi(num_workers=0)

# 2. 추출할 의미 있는 품사 정의
MEANINGFUL_TAGS = {'NNG', 'NNP', 'VV', 'VA', 'XR', 'SL'}

def tokenize_csv_title_content(input_path, output_csv, vocab_csv):
    word_counter = Counter()
    processed_results = []

    if not os.path.exists(input_path):
        print(f"오류: 파일을 찾을 수 없습니다: {input_path}")
        return

    print("CSV 파일 로드 중...")
    # CSV 파일을 읽어옵니다. (인코딩 에러 방지를 위해 utf-8-sig 사용)
    df_raw = pd.read_csv(input_path, encoding='utf-8-sig')

    print("제목 및 본문 의미 단어 추출 중 (Kiwi)...")
    # DataFrame의 각 행을 순회합니다.
    for _, row in tqdm(df_raw.iterrows(), total=len(df_raw)):
        try:
            # 제목과 본문 컬럼 데이터 가져오기 (컬럼명이 다를 경우 여기서 수정)
            title = str(row.get('title', ''))
            content = str(row.get('content', ''))
            date = str(row.get('date', ''))
            
            texts = [('제목', title), ('본문', content)]

            for content_type, text in texts:
                if not text or len(text.strip()) < 2:
                    continue
                
                # Kiwi 토큰화
                tokens = kiwi.tokenize(text)
                
                # 의미 있는 품사만 필터링
                meaningful_words = [t.form for t in tokens if t.tag in MEANINGFUL_TAGS]
                
                # 단어 빈도수 업데이트
                word_counter.update(meaningful_words)
                
                # 결과 정리
                meaningful_str = " ".join(meaningful_words)
                processed_results.append({
                    "date": date,
                    "type": content_type,
                    "original_text": text.replace("\n", " "),
                    "meaningful_words": meaningful_str
                })
        except Exception as e:
            continue

    # 3. 결과 저장
    if processed_results:
        # 추출 결과 저장
        res_df = pd.DataFrame(processed_results)
        res_df.to_csv(output_csv, index=False, encoding="utf-8-sig")
        
        # 단어 빈도 저장
        vocab_df = pd.DataFrame(word_counter.most_common(), columns=['word', 'count'])
        vocab_df.to_csv(vocab_csv, index=False, encoding="utf-8-sig")
        
        print(f"\n[완료]")
        print(f"- 추출 결과 저장됨: {output_csv}")
        print(f"- 단어 빈도 리스트 저장됨: {vocab_csv}")
        print(f"- 고유 의미 단어 수: {len(vocab_df)}개")
    else:
        print("\n[알림] 추출된 데이터가 없습니다.")

# 실행 설정
# 업로드하신 posts_samsung_with_comment.csv 파일 경로로 설정하세요.
INPUT_CSV = "../data/posts_hynix_with_comment.csv" 
RESULT_CSV = "fmkorea_meaningful_results_csv.csv"
VOCAB_CSV = "fmkorea_word_counts_csv.csv"

tokenize_csv_title_content(INPUT_CSV, RESULT_CSV, VOCAB_CSV)

C:\Users\Jeon\AppData\Local\Temp\ipykernel_26068\2390522692.py:8: DeprecationWarning: behavior of `num_workers=0` is changed since v0.21.0. If you want to keep the previous behavior, please set `num_workers=-1`.
  kiwi = Kiwi(num_workers=0)


CSV 파일 로드 중...
제목 및 본문 의미 단어 추출 중 (Kiwi)...


100%|██████████| 3367/3367 [00:02<00:00, 1318.53it/s]


[완료]
- 추출 결과 저장됨: fmkorea_meaningful_results_csv.csv
- 단어 빈도 리스트 저장됨: fmkorea_word_counts_csv.csv
- 고유 의미 단어 수: 6547개
